In [1]:
import sys
sys.path.insert(0, 'D:\MyApps\ArcGIS Pro 3.2\Resources\ArcPy')
import csv
import arcpy
import datetime
from collections import defaultdict

In [2]:
#SETTING THINGS UP 
# Set the workspace to your geodatabase
arcpy.env.workspace = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb"

# Name of the Trace Network
trace_network_name = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/newfeaturedata/trace_20mnoshore"

# Name of the stations feature class and the field that contains the network global IDs
old_stations_featureclass = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/water_stations_snap20info"
network_global_id_field = "GlobalID"
network_feature_class = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/newfeaturedata/EnhancedWater__noshore_1"

In [ ]:
#1A prepp
#stations are already snapped, now join the attribute steam_order from the watercourse featureclass to these stations that are closest

# Output feature class for the joined result
#stations_featureclass = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/stations_with_stream_order"

# Perform the spatial join
arcpy.analysis.SpatialJoin(
    target_features=old_stations_featureclass,
    join_features=network_feature_class,
    out_feature_class=stations_featureclass,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="INTERSECT",
    search_radius="",  # You can specify a search radius if necessary
    distance_field_name="DistanceToRoute")  # You can specify a field to store the join distance if necessary

print("Spatial join completed. Stream order joined to stations.")

In [4]:
#set up functions needed for the loop


def select_intersected_stations(traced_routes, stations_featureclass, route_id):
    """
    Select stations intersected by the traced routes.
    :param traced_geometries: List of geometries representing traced routes.
    :param stations_featureclass: The feature class of the stations.
    :param route_id: The unique identifier for the route.
    :return: Dictionary with route ID as key and list of intersected station IDs as value.
    """
    intersected_stations = defaultdict(list)
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

    # Create an in-memory layer from the traced route geometries
    traced_routes_layer = f"in_memory//tracedRoutes_{timestamp}"
    arcpy.CopyFeatures_management(traced_routes, traced_routes_layer)

    # Create an in-memory layer from the stations
    stations_layer = f"in_memory//stations_{timestamp}"
    arcpy.MakeFeatureLayer_management(stations_featureclass, stations_layer)

    # Perform the spatial join
    spatial_join_result = f"in_memory//spatialJoinResults_{timestamp}"
    arcpy.SpatialJoin_analysis(target_features = traced_routes_layer, 
                               join_features = stations_layer, 
                               out_feature_class = spatial_join_result, 
                               join_operation = "JOIN_ONE_TO_MANY", 
                               join_type = "KEEP_COMMON", 
                               match_option="INTERSECT")
    #add a route_id field to the spatialjoin featureclass
    arcpy.AddField_management(spatial_join_result, "route_id", "LONG")

    with arcpy.da.UpdateCursor(spatial_join_result, ["route_id"]) as cursor:
        for row in cursor:
            row[0] = route_id
            cursor.updateRow(row)

    # Gather intersected station IDs and associate them with the route_id
    with arcpy.da.SearchCursor(spatial_join_result, ["STATION"]) as cursor:
        for row in cursor:
            station_id = row[0]
            intersected_stations[route_id].append(station_id)

            
    # Clean up in-memory layers
    arcpy.Delete_management(traced_routes_layer)
    arcpy.Delete_management(stations_layer)
    arcpy.Delete_management(spatial_join_result)

    return intersected_stations






In [6]:
processed_stations = set()
route_id_counter = 1


#SETTING THINGS UP 
# Set the workspace to your geodatabase
arcpy.env.workspace = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb"

# Name of the Trace Network
trace_network_name = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/newfeaturedata/trace_20mnoshore"

# Name of the stations feature class and the field that contains the network global IDs
stations_featureclass = "C:/Users/sonne/Documents/ArcGIS/Projects/MyProject/MyProject.gdb/SW_stations"
network_global_id_field = "GlobalID"

# Name of the output routes feature class
routes_featureclass = "forlooptraced_routes_SW"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, routes_featureclass, "POLYLINE")

route_field_names = [field.name for field in arcpy.ListFields(routes_featureclass)]

# Check if 'station_list_field' exists, create it if not
if 'station_list_field' not in route_field_names:
    arcpy.AddField_management(routes_featureclass, 'station_list_field', 'TEXT')

# Now you can safely use 'station_list_field' in your script


# Check if the routes feature class exists, if not create it

# Add a field to store the route_id if not exists
route_id_field = "route_id"
if len(arcpy.ListFields(routes_featureclass, route_id_field)) == 0:
    arcpy.AddField_management(routes_featureclass, route_id_field, "LONG")



In [9]:
# Check if the dataset exists, and delete it if it does
if arcpy.Exists("Trace_Results_Aggregated_Points"):
    arcpy.Delete_management("Trace_Results_Aggregated_Points")

if arcpy.Exists("in_memory//DissolvedAggregatedLines"):
    arcpy.Delete_management("in_memory//DissolvedAggregatedLines")

#delete 

In [5]:
stations_featureclass = "D:\MyApps\ArcGIS Pro 3.2\greenbelt_gis\Input\MyProject1.gdb\SW_stations"

In [ ]:
import datetime
# Check if the dataset exists, and delete it if it does
if not arcpy.Exists("in_memory//tempFC"):
    temp_fc = arcpy.CreateFeatureclass_management("in_memory", "tempFC", 
                                              "POINT", spatial_reference=arcpy.SpatialReference(3161))
station_to_route_mapping = defaultdict(set)



# Loop through each station and perform a downstream trace
with arcpy.da.SearchCursor(stations_featureclass, ["OID@", "SHAPE@", "STATION"]) as stations_cursor:
    for station in stations_cursor:
        station_oid = station[0]  # Object ID of the station at index 0
        station_shape = station[1]  # Geometry of the station at index 2
        station_id = station[2]  # Station ID of the station at index 3
        # Skip the station if it has already been processed
        if station_id  in processed_stations:
            continue
        route_id = route_id_counter


        # Insert the current station into the temporary feature class
        with arcpy.da.InsertCursor(temp_fc, ["SHAPE@"]) as insert_cursor:
            insert_cursor.insertRow([station_shape])
        # Confirm that the feature class has been populated
        feature_count = arcpy.GetCount_management(temp_fc)[0]
        if feature_count == '0':
            raise RuntimeError("Temporary feature class is empty.")
        
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        # Unique name for each iteration
        aggregated_lines_fc = f"aggregatedLines_{timestamp}"
        aggregated_points_fc = f"aggregatedPoints_{timestamp}"

        # Before running the trace, add a print statement
        print(f"Attempting trace from station with station_id: {station_id}") 
        # Perform the trace
        trace_result = arcpy.tn.Trace(
            in_trace_network=trace_network_name,
            trace_type="DOWNSTREAM",
            starting_points=temp_fc,
            result_types = "AGGREGATED_GEOMETRY",
            clear_all_previous_trace_results = "CLEAR_ALL_PREVIOUS_TRACE_RESULTS",
            aggregated_lines = aggregated_lines_fc,
            aggregated_points = aggregated_points_fc
            )

        # Dissolve the aggregated lines
        dissolved_aggregated_lines_fc = f"in_memory//DissolvedAggregatedLines_{timestamp}"
        arcpy.Dissolve_management(in_features=aggregated_lines_fc, 
                                out_feature_class=dissolved_aggregated_lines_fc,
                                # Add additional parameters as needed (e.g., dissolve_field, statistics_fields)
                                )

       
        # Append the dissolved lines to the existing routes feature class
        arcpy.Append_management(inputs=[dissolved_aggregated_lines_fc], target=routes_featureclass, schema_type="NO_TEST")
        

        # use the generated route to select stations based on spatial location
        intersected_stations = select_intersected_stations(dissolved_aggregated_lines_fc, stations_featureclass, route_id)
        
        # The keys of the 'intersected_stations' dictionary are the route IDs and the values are lists of station IDs
        for rid, station_ids in intersected_stations.items():
            for station_id in station_ids:
                # Append the current route_id to the list of route_ids for the intersected station
                station_to_route_mapping[station_id].add(rid)

        # Read the geometry from the dissolved feature class
        with arcpy.da.SearchCursor(dissolved_aggregated_lines_fc, ["SHAPE@"]) as search_cursor:
            for row in search_cursor:
                dissolved_geometry = row[0]  # Get the geometry from the row

        # Insert the dissolved geometry into the routes feature class
        with arcpy.da.InsertCursor(routes_featureclass, ['SHAPE@', route_id_field]) as insert_cursor:
            insert_cursor.insertRow([dissolved_geometry, route_id])

        # Update the routes feature class with the list of intersected stations
        with arcpy.da.UpdateCursor(routes_featureclass, [route_id_field, "station_list_field"]) as routes_cursor:
            for route in routes_cursor:
                if route[0] == route_id:
                    route[1] = ','.join(map(str, intersected_stations[route_id]))
                    routes_cursor.updateRow(route) 

        # Remove the station's point from the temporary feature class
        with arcpy.da.UpdateCursor(temp_fc, ["OID@"]) as delete_cursor:
            for row in delete_cursor:
                delete_cursor.deleteRow()

        #remove intersected parts from the selection set
        processed_stations.add(station_id)

        #remove the route feature class generated and the dissolved
        arcpy.Delete_management(aggregated_lines_fc)
        arcpy.Delete_management(aggregated_points_fc)
        arcpy.Delete_management(dissolved_aggregated_lines_fc)


        # Increment the route ID counter
        route_id_counter += 1

# Clean up: Delete the temporary feature class
arcpy.Delete_management(temp_fc)       

In [ ]:
station_to_route_mapping

In [3]:
import json

# Convert the dictionary to a JSON string
# json_str = json.dumps(station_to_route_mapping)

# # Write to a file
# with open("station_to_route_mapping.json", "w") as file:
#     file.write(json_str)
with open("station_to_route_mapping.json", "w") as file:
    json.dump(station_to_route_mapping, file)


NameError: name 'station_to_route_mapping' is not defined

In [2]:


# # Read the file
# with open("station_to_route_mapping.json", "r") as file:
#     json_str = file.read()

# # Convert JSON string back to dictionary
# loaded_dict = json.loads(json_str)

# print(loaded_dict)

import json

with open('station_to_route_mapping.json') as f:
  data = json.load(f)

loaded_dict = data

In [3]:
stations_featureclass = "D:\MyApps\ArcGIS Pro 3.2\greenbelt_gis\Input\MyProject1.gdb\SW_stations"

In [6]:
# Delete the existing 'route_id' field
arcpy.DeleteField_management(stations_featureclass, "route_id")

# Add the 'route_id' field as a TEXT field
arcpy.AddField_management(stations_featureclass, "route_id", "TEXT")

<Result 'D:\\MyApps\\ArcGIS Pro 3.2\\greenbelt_gis\\Input\\MyProject1.gdb\\SW_stations'>

In [4]:

print(json.dumps(loaded_dict, indent=4))

{
    "3007703502.0": [
        1,
        32,
        134,
        139
    ],
    "3007704002.0": [
        1,
        5,
        32,
        134,
        139
    ],
    "3007701302.0": [
        1,
        2,
        3,
        4,
        5,
        6,
        29,
        30,
        31,
        32,
        33,
        34,
        126,
        127,
        128,
        129,
        130,
        131,
        132,
        133,
        134,
        135,
        136,
        137,
        138,
        139,
        140,
        141,
        142,
        143,
        144,
        145,
        146,
        147,
        148,
        149,
        150
    ],
    "3007702202.0": [
        1,
        2,
        3,
        4,
        5,
        6,
        29,
        30,
        32,
        33,
        34,
        126,
        127,
        128,
        129,
        130,
        131,
        132,
        133,
        134,
        135,
        136,
        137,
        139,
        140,
        141,

In [17]:
row

[16018410402.0, None]

In [5]:
# Update the 'route_id' field with comma-separated route IDs
with arcpy.da.UpdateCursor(stations_featureclass, ["STATION", "route_id"]) as cursor:
    for row in cursor:
        if str(row[0]) in loaded_dict:
            print('>', end='')
            row[1] = ','.join(map(str, loaded_dict[str(row[0])]))
            cursor.updateRow(row)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [10]:
#print station_featureclass field names
field_names = [f.name for f in arcpy.ListFields(stations_featureclass)]
print(field_names)

['OBJECTID', 'Shape', 'Join_Count', 'DistanceToRoute', 'TARGET_FID', 'Join_Count_1', 'TARGET_FID_1', 'STATION', 'NAME', 'DESCRIPTION', 'LATITUDE', 'LONGITUDE', 'STATUS', 'FIRST_YEAR', 'LAST_YEAR', 'TOTAL_YEARS', 'MISS_YEARS', 'OGF_ID', 'WCRS_TYPE', 'OFF_NAME', 'FLOW_CLASS', 'EFF_DATE', 'GEO_UPD_DT', 'SessionName', 'Enabled', 'FNODE', 'TNODE', 'STRAHLER', 'SHREVE', 'FLOWDIRECTION', 'FLOWDIRECTION_1', 'FLOWDIRECTION_2', 'route_id']


In [11]:
stations_featureclass

'D:\\MyApps\\ArcGIS Pro 3.2\\greenbelt_gis\\Input\\MyProject1.gdb\\SW_stations'

Now try to do the rest with the current ouput (First part finished)

In [12]:
arcpy.env.workspace = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb"
stations_featureclass_update= "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/stations_with_route_id"
#arcpy.AddField_management(stations_featureclass_update, 'route_id', 'TEXT')


In [13]:
# use a copy of routes without nulls - note: find a way to clean it with script
routes_featureclass_update = "D:/MyApps/ArcGIS Pro 3.2/greenbelt_gis/Input/MyProject1.gdb/forlooptraced_routes_nona"

Third part: Greenbelt Identify if the Routes Intersect the Greenbelt:
For each route, determine if it intersects with the greenbelt area and flag it accordingly.

Calculate the Distance of Each Station to the Greenbelt:
For stations that are near or within the greenbelt, calculate the distance to the nearest greenbelt boundary.

Determine the Segment of the Route Within the Greenbelt:
For routes that intersect the greenbelt, calculate the length of the route segment that lies within the greenbelt.